In [1]:
#from portfolio import Asset, Portfolio
from pandas import Timestamp as tmpstemp
from pandas import Timedelta as tmpdelta
import pandas as pd
import os
from coinbase.rest import RESTClient
import json

In [2]:
if os.environ['USERDOMAIN'] == 'PC-MNR-LEBEDEVA':
    with open('coinbase_cloud_api_key.json') as f:
        d = json.load(f)
    os.environ['API_KEY'] =d['name']
    os.environ['API_SECRET'] = d['privateKey']

In [3]:
client = RESTClient(api_key = os.environ['API_KEY'], api_secret=os.environ['API_SECRET'])

client.get_accounts()['accounts']

[{'uuid': 'eabe46ba-e374-5e79-9511-b8a02526a38f',
  'name': 'USDT Wallet',
  'currency': 'USDT',
  'available_balance': {'value': '709.5778905315224', 'currency': 'USDT'},
  'default': True,
  'active': True,
  'created_at': '2024-01-06T01:58:06.277Z',
  'updated_at': '2024-05-03T01:00:31.134Z',
  'deleted_at': None,
  'type': 'ACCOUNT_TYPE_CRYPTO',
  'ready': True,
  'hold': {'value': '0', 'currency': 'USDT'},
  'retail_portfolio_id': '24485da6-2523-5cdc-8578-214172d3bcf2'},
 {'uuid': 'e1c03406-edee-5930-97e7-e2062c2a0769',
  'name': 'ETH2 Wallet',
  'currency': 'ETH2',
  'available_balance': {'value': '0', 'currency': 'ETH2'},
  'default': True,
  'active': True,
  'created_at': '2023-07-18T21:57:52.811Z',
  'updated_at': '2023-07-18T21:57:52.811Z',
  'deleted_at': None,
  'type': 'ACCOUNT_TYPE_CRYPTO',
  'ready': False,
  'hold': {'value': '0', 'currency': 'ETH2'},
  'retail_portfolio_id': '24485da6-2523-5cdc-8578-214172d3bcf2'},
 {'uuid': '76da3424-8129-5cb3-bcd1-c8c5537d6b7f',
  '